# Producción de madera

Basado en: *[Business Optimizaiton Using Mathematical Programming](https://link.springer.com/book/10.1007/978-3-030-73237-0)* pp. 47

Una empresa maderera dispone de dos tipos de madera, pino y abedul, que pueden utilizarse para fabricar dos productos: madera contrachapada o tablero. Para producir 100 m2 de madera contrachapada se necesitan 300 m2 de pino y 300 m2 de abedul, mientras que para producir 100 m2 de tablero se necesitan 100 m2 de pino y 500 m2 de abedul. La empresa tiene acceso a 5000 m2 de pino y 6000 m2 de abedul en cada período de trabajo. Se pueden estimar los costes e ingresos de las diferentes materias primas y productos, y la empresa desea crear un modelo de LP para maximizar los ingresos netos por período. La empresa quiere producir al menos 500 m2 de cada producto por período.

### Índices
- $m \in \{p,a\}$: tipos de madera, pino y abedul
- $r \in \{c,t\}$: tipos de productos, madera contrachapada y tableros

### Variables de decisión
- $c_{m, r}$: cantidad de madera $m$ a utilizar en la producción del producto $r$
- $x_{r}$: cantidad de producto $r$ a producir

### Restricciones
- $x_{r} \geq 500 \quad \forall p$: mínimo de producción
- $\sum_{r} c_{m, r} \leq D_{m} \quad \forall p$: límite de madera disponible
- $3 x_{c} - c_{p, c} = 0$: pino para producir madera contrachapada
- $3 x_{c} - c_{a, c} = 0$: abedul para producir madera contrachapada
- $x_{t} - c_{p, t} = 0$:  pino para producir tableros
- $5 x_{t} - c_{a, t} = 0$: abedul para producir tableros

### Parámetros
- $CV_{m, r}$: costo de la madera $m$ para producir el producto $r$
- $P_{r}$: precio de venta del producto $r$
- $D_{m}$: disponibilidad de la madera $m$

### Función Objetivo
- $\max \sum_{r} P_{r} x_{r} - \sum_{m, r} CV_{m} c_{m, r}$

In [9]:
# Implementación en Python

import pyomo.environ as pyo

# diccionario con los parámetros del modelo
parametros = {
    # costos de la madera
    'CV': {
        'p': 30,
        'a': 50
    },
    # precios de venta de los productos
    'PV': {
        'c': 1000,
        't': 1200,
    },
    # disponibilidad de madera
    'D': {
        'p': 5000,
        'a': 6000,
    },
    # cantidad mínima que se espera producir
    'MinProd': {
        'c': 500,
        't': 500
    },
    # cantidad de tipo madera necesaria por unidad de producto
    'MatProd': {
        ('p', 'c'): 3,
        ('a', 'c'): 3,
        ('p', 't'): 1,
        ('a', 't'): 5
    }
}

In [10]:
# Creemos una función que construya al modelo

def modelo_madera(param: dict):
    
    # creamos el modelo
    modelo = pyo.ConcreteModel(name= 'productos de madera')
    
    # especificamos los índices
    modelo.m = pyo.Set(initialize= ['p', 'a'])
    modelo.r = pyo.Set(initialize= ['c', 't'])
    
    # parámetros del modelo, nótese cómo Pyomo utiliza diccionarios automáticamente para especificar los parámetros
    
    # costo de la madera
    modelo.CV = pyo.Param(modelo.m, initialize= param['CV'])
    
    # precio de venta de los productos
    modelo.PV = pyo.Param(modelo.r, initialize= param['PV'])
    
    # madera necesaria por unidad de producto
    modelo.MatProd = pyo.Param(modelo.m, modelo.r, initialize= param['MatProd'])
    
    
    # variables de decisión
    
    # cantidad de madera a utilizar en la producción de cada tipo de producto
    modelo.c = pyo.Var(modelo.m, modelo.r, domain= pyo.NonNegativeReals)
    # cantidad a producir por tipo de producto
    modelo.x = pyo.Var(modelo.r, domain= pyo.NonNegativeReals)
    
    
    # las restricciones en Pyomo se especifican como funciones, los argumentos corresponden a los índices de 
    # las variales y parámetros usados en la restricción, en este caso, se necesita una restricción 
    # para cada tipo de madera, no es necesario especificar los índices de los productos 
    # ya que estos se suman en la restricción
    # IMPORTANTE: el primer argumento siempre es el modelo que se está construyendo
    def limite_madera(mod, m):
        return sum(mod.c[m, r] for r in modelo.r) <= param['D'][m]
    # rule corresponde al nombre de la función que chequea la restricción
    modelo.rest_limite_madera = pyo.Constraint(modelo.m, rule= limite_madera)
    
    
    # restricciones de producción mínima
    def min_prod(mod, r):
        return mod.x[r] >= param['MinProd'][r]
    modelo.rest_min_prod = pyo.Constraint(modelo.r, rule= min_prod)
    
    # balance para cada tipo madera
    def balance_madera(mod, m, r):
        return mod.MatProd[m, r] * mod.x[r] - mod.c[m, r] == 0
    modelo.rest_balance_madera = pyo.Constraint(modelo.m, modelo.r, rule= balance_madera)
    
    # función objetivo
    def objetivo(mod):
        return sum(mod.PV[r] * mod.x[r] - sum(mod.CV[m] * mod.c[m, r] for m in mod.m) for r in mod.r)
    modelo.obj = pyo.Objective(rule= objetivo, sense= pyo.maximize)
    
    return modelo
    

In [11]:
# ahora creamos una instancia del modelo y lo ejecutamos

# creamos el modelo utilizando los parámetros definidos
modelo = modelo_madera(parametros)

# utilizamos un solver
solver = pyo.SolverFactory('gurobi')
# resolvemos el modelo y guardamos los resultados
resultados = solver.solve(modelo)
# la siguiente instrucción genera un error en caso de que no se haya encontrado una solución óptima
pyo.assert_optimal_termination(resultados)

# imprimimos los resultados
modelo.display()

Model productos de madera

  Variables:
    c : Size=4, Index=c_index
        Key        : Lower : Value  : Upper : Fixed : Stale : Domain
        ('a', 'c') :     0 : 3500.0 :  None : False : False : NonNegativeReals
        ('a', 't') :     0 : 2500.0 :  None : False : False : NonNegativeReals
        ('p', 'c') :     0 : 3500.0 :  None : False : False : NonNegativeReals
        ('p', 't') :     0 :  500.0 :  None : False : False : NonNegativeReals
    x : Size=2, Index=r
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          c :     0 : 1166.6666666666667 :  None : False : False : NonNegativeReals
          t :     0 :              500.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1346666.6666666667

  Constraints:
    rest_limite_madera : Size=2
        Key : Lower : Body   : Upper
          a :  None : 6000.0 : 6000.0
          p :  None : 4000

## Obtención de resultados

Una de las principales ventajas de utilizar un lenguaje como Python en la implementación de modelos de optimización es que se puede integrar fácilmente a flujos de trabajo sin la necesidade de interfaces de intercambio de información, y manteniendo todo dentro de la misma plataforma tecnológica y utilizando el mismo código fuente.


In [12]:
# la función principal para obtener valores de las variables de decisión es value()

# cantidad de madera a utilizar en la producción de cada tipo de producto
for m in modelo.m:
    for r in modelo.r:
        print(f'Cantidad de madera {m} para producir producto {r} = {pyo.value(modelo.c[m, r])}')

Cantidad de madera p para producir producto c = 3500.0
Cantidad de madera p para producir producto t = 500.0
Cantidad de madera a para producir producto c = 3500.0
Cantidad de madera a para producir producto t = 2500.0


In [13]:
# exploremos todas las variables del modelo

for v in modelo.component_objects(pyo.Var):
    print(f'Variable: {v}')
    for index in v:
        print(f'\t índice: {index}', pyo.value(v[index]))

Variable: c
	 índice: ('p', 'c') 3500.0
	 índice: ('p', 't') 500.0
	 índice: ('a', 'c') 3500.0
	 índice: ('a', 't') 2500.0
Variable: x
	 índice: c 1166.6666666666667
	 índice: t 500.0


In [14]:
# generémoslo de manera dinámica utilizando un diccionario
dict_resultados = dict()

# la siguiente instrucción itera sobre todas las variables del modelo y crea un diccionario para cada una
dict_resultados = {v.name: {index: pyo.value(v[index]) for index in v} for v in modelo.component_objects(pyo.Var)}
dict_resultados

{'c': {('p', 'c'): 3500.0,
  ('p', 't'): 500.0,
  ('a', 'c'): 3500.0,
  ('a', 't'): 2500.0},
 'x': {'c': 1166.6666666666667, 't': 500.0}}

In [15]:
# ahora creemos un Dataframe para cada conjunto de variables de decisión

import pandas as pd

dict_df = {v: pd.DataFrame.from_dict(dict_resultados[v], orient='index').reset_index(drop=False, inplace=False) for v in dict_resultados}
dict_df['c']

,index,0
0,"(p, c)",3500.0
1,"(p, t)",500.0
2,"(a, c)",3500.0
3,"(a, t)",2500.0


In [16]:
# vemos que para la cantidad de material el índice es una tupla, para facilitar el trabajo, dividamos cada dimensión en una columna

# primero transformamos las tuplas en multiíndices
dict_df['c'].set_index(pd.MultiIndex.from_tuples(dict_df['c']['index'].tolist()), inplace=True)
dict_df['c']

index       0
p c  (p, c)  3500.0
  t  (p, t)   500.0
a c  (a, c)  3500.0
  t  (a, t)  2500.0

In [17]:
# ahora reiniciamos los índices
dict_df['c'].reset_index(drop=False, inplace=True)
dict_df['c']

,level_0,level_1,index,0
0,p,c,"(p, c)",3500.0
1,p,t,"(p, t)",500.0
2,a,c,"(a, c)",3500.0
3,a,t,"(a, t)",2500.0


In [18]:
# eliminamos la columna 'index'
dict_df['c'].drop(columns=['index'], inplace=True)
dict_df['c']

,level_0,level_1,0
0,p,c,3500.0
1,p,t,500.0
2,a,c,3500.0
3,a,t,2500.0


In [19]:
# y finalmente le cambiamos el nombre
dict_df['c'].columns = ['tipo_madera', 'tipo_producto', 'cantidad']
dict_df['c']

,tipo_madera,tipo_producto,cantidad
0,p,c,3500.0
1,p,t,500.0
2,a,c,3500.0
3,a,t,2500.0


In [21]:
# para la variable cantidad de productos sólo necesitamos cambiar los nombres
dict_df['x'].columns = ['tipo_producto', 'cantidad']
dict_df['x']

,tipo_producto,cantidad
0,c,1166.666667
1,t,500.000000
